In [ ]:
import pandas as pd
import numpy as np
import spacy
import string
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import gensim.downloader as api
from gensim.models import KeyedVectors

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/MINI PROJECT/DATASET/augmented_dataset1.csv')
df.head()

,PID,Text data,Label
0,0,waiting for my mind to have breakdown once the...,1
1,1,for my new anymore little bit not of just and ...,1
2,2,new year feeling there else depressed last eve...,1
3,3,for my to have the new feeling know about anyo...,1
4,4,to the new year in start and into great myself...,1


In [ ]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
model=api.load('glove-twitter-100')

In [ ]:
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower().strip() for word in doc]

    # Removing stop words and punctuations
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

    # Return preprocessed list of tokens
    return mytokens

In [ ]:
df['tokens'] = df['Text data'].apply(spacy_tokenizer)


In [ ]:
# Generate document vectors for each document
def document_vector(tokens, embeddings, dim):
    token_vectors = [embeddings[token] for token in tokens if token in embeddings]
    if not token_vectors:
        return np.zeros(dim)
    return np.mean(token_vectors, axis=0)

In [ ]:
df['vec'] = df['tokens'].apply(lambda x: document_vector(x, model, model.vector_size))

In [ ]:
# Prepare features (X) and labels (y)
X = np.vstack(df['vec'])
y = df['Label']

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [ ]:
model_pipeline_lr = Pipeline([
    ('lr', LogisticRegression(max_iter=1000, random_state=1))  # Increase max_iter if needed
])


In [ ]:
# Train the model
model_pipeline_lr.fit(X_train, y_train)

# Predict on the test set
y_pred_lr = model_pipeline_lr.predict(X_test)


In [ ]:
# Evaluate the model
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr, average='weighted')
recall_lr = recall_score(y_test, y_pred_lr, average='weighted')
f1_score_lr = f1_score(y_test, y_pred_lr, average='weighted')
classification_report_lr = classification_report(y_test, y_pred_lr)

# Print evaluation metrics
print("Evaluation Metrics for Logistic Regression Model")
print("------------------------------------------------")
print(f"Accuracy: {accuracy_lr:.4f}")
print(f"Precision: {precision_lr:.4f}")
print(f"Recall: {recall_lr:.4f}")
print(f"F1-score: {f1_score_lr:.4f}")
print("Classification Report:")
print(classification_report_lr)

Evaluation Metrics for Logistic Regression Model
------------------------------------------------
Accuracy: 0.6556
Precision: 0.6563
Recall: 0.6556
F1-score: 0.6543
Classification Report:
              precision    recall  f1-score   support

           0       0.69      0.65      0.67      1194
           1       0.64      0.58      0.61      1236
           2       0.64      0.74      0.69      1182

    accuracy                           0.66      3612
   macro avg       0.66      0.66      0.65      3612
weighted avg       0.66      0.66      0.65      3612



In [5]:
import pandas as pd
import numpy as np
import spacy
import string
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import gensim.downloader as api

# Load dataset
df = pd.read_csv("/content/drive/MyDrive/MINI PROJECT/DATASET/augmented_dataset1.csv")

# Load SpaCy model and define stop words and punctuations
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
punctuations = string.punctuation

# Load pre-trained GloVe embeddings
model = api.load('glove-twitter-100')

# Function to tokenize and preprocess text
def spacy_tokenizer(sentence):
    doc = nlp(sentence)
    mytokens = [word.lemma_.lower().strip() for word in doc]
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]
    return mytokens

# Apply tokenization to each row in 'Text data' column
df['tokens'] = df['Text data'].apply(spacy_tokenizer)

# Generate document vectors for each document
def document_vector(tokens, embeddings, dim):
    token_vectors = [embeddings[token] for token in tokens if token in embeddings]
    if not token_vectors:
        return np.zeros(dim)
    return np.mean(token_vectors, axis=0)

df['vec'] = df['tokens'].apply(lambda x: document_vector(x, model, model.vector_size))

# Prepare features (X) and labels (y)
X = np.vstack(df['vec'])
y = df['Label']

# Normalize features for Naive Bayes
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size=0.8, random_state=1)

# Initialize and train Naive Bayes model
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

# Predict on the test set
y_pred_nb = naive_bayes.predict(X_test)

# Evaluate the model
accuracy_nb = accuracy_score(y_test, y_pred_nb)
precision_nb = precision_score(y_test, y_pred_nb, average='weighted')
recall_nb = recall_score(y_test, y_pred_nb, average='weighted')
f1_score_nb = f1_score(y_test, y_pred_nb, average='weighted')
classification_report_nb = classification_report(y_test, y_pred_nb)

# Print evaluation metrics
print("Evaluation Metrics for Naive Bayes Model")
print("------------------------------------------------")
print(f"Accuracy: {accuracy_nb:.4f}")
print(f"Precision: {precision_nb:.4f}")
print(f"Recall: {recall_nb:.4f}")
print(f"F1-score: {f1_score_nb:.4f}")
print("Classification Report:")
print(classification_report_nb)


[==================================================] 100.0% 387.1/387.1MB downloaded
Evaluation Metrics for Naive Bayes Model
------------------------------------------------
Accuracy: 0.5648
Precision: 0.5774
Recall: 0.5648
F1-score: 0.5610
Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.49      0.56      1194
           1       0.57      0.49      0.52      1236
           2       0.52      0.72      0.60      1182

    accuracy                           0.56      3612
   macro avg       0.58      0.57      0.56      3612
weighted avg       0.58      0.56      0.56      3612

